In [302]:
using Pkg
Pkg.activate(".")

  Activating environment at `~/CMU-Courses/Optimal-Control/Project/Project.toml`


In [303]:
using RobotDynamics
using LinearAlgebra
using StaticArrays
using Altro
using TrajectoryOptimization
const TO = TrajectoryOptimization
const RD = RobotDynamics
using ForwardDiff  # needed for @autodiff
using FiniteDiff   # needed for @autodiff
import RobotDynamics:AbstractModel

In [304]:
#struct Rocket{T} <: AbstractModel

struct Rocket{T} <: RD.ContinuousDynamics 
    m::T
    omega::SVector{3,T}
    α::T
    g::SVector{3,T}
end

In [305]:
#hat matrix operator 

function skewmatrix(x)
    S = SA[0 -x[3] x[2]; x[3] 0 -x[1]; -x[2] x[1] 0]
    return S
end

skewmatrix (generic function with 1 method)

In [306]:
#Dynamics including mass in state

function RD.dynamics(model::Rocket, x, u)
    m = model.m   # intial mass of the rocket
    omega = model.omega   # mass of the pole j(point mass at the end) in kg
    α = model.α     # length of the pole in m
    g = model.g     # gravity m/s^2

    #q  = x[SA[1,2,3]] #position
    #qd = x[SA[4,5,6]] #first derivative is the velocity, last elements of state
    
    upper = [(@SMatrix zeros(3,3)) (SMatrix{3,3}(1I)) (@SVector zeros(3))]
    #mid = [(@SMatrix zeros(3,3)) (SMatrix{3,3}(1I)) (@SVector zeros(3))]
    mid = [(-(skewmatrix(omega))^2) (-2*skewmatrix(omega)) (@SVector zeros(3))]
    lower = (@SVector zeros(7))'
    
    A = [upper; mid; lower] #makes A static
    
    #B = [SMatrix{3,3}(1I); @SMatrix zeros(3,3); (@SVector zeros(3))'] #make B static
    
    B = [@SMatrix zeros(3,4); SMatrix{3,3}(1I) @SVector zeros(3); (@SVector zeros(3))' -α] #make B static
    
    B_2 = [(g+(u[SA[1,2,3]]/x[7])); norm(u[SA[1,2,3]])]
    
    xdot = A*x + B*B_2

    return xdot
end

RD.state_dim(::Rocket) = 7
RD.control_dim(::Rocket) = 4

In [307]:
# function massdynamics(model::Rocket, u)
#     α = model.α 
#     mdot =  -α*norm(u)
# end

In [308]:
# function massRK4(model::Rocket, t)
#     m = model.m
    
# end

In [309]:
# #Dynamics with no mass in state

# #import RobotDynamics:dynamics

# function RD.dynamics(model::Rocket, x, u)
    
#     m_rocket = model.m   # intial mass of the rocket
#     omega = model.omega   # mass of the pole j(point mass at the end) in kg
#     α = model.α     # length of the pole in m
#     g = model.g     # gravity m/s^2

#     #q  = x[SA[1,2,3]] #position
#     #qd = x[SA[4,5,6]] #first derivative is the velocity, last elements of state
    
#     upper = [(@SMatrix zeros(3,3)) (SMatrix{3,3}(1I))]
#     #mid = [(@SMatrix zeros(3,3)) (SMatrix{3,3}(1I)) (@SVector zeros(3)]
#     mid = [-(skewmatrix(omega))^2 -2*skewmatrix(omega)]
#     #lower = (@SVector zeros(7))'
    
#     #mdot = -α*norm(u)
    
#     #println("this is time: ", t)
#     #currentmass = m_rocket - mdot*t
    
#     #currentmass = m_rocket
#     #println("this is current mass: ", currentmass)
    
#     A = [upper; mid] #makes A static
    
#     #B = [SMatrix{3,3}(1I); @SMatrix zeros(3,3); (@SVector zeros(3))'] #make B static
    
#     B = [@SMatrix zeros(3,3); SMatrix{3,3}(1I)] #make B static
    
#     d = (g+(u/m))
    
#     xdot = A*x + B*d
    
#     #println(typeof(xdot))

#     return xdot
# end

# RD.state_dim(::Rocket) = 6
# RD.control_dim(::Rocket) = 3

In [310]:
#Model and discretization
T = Float64
initialmass = 2000.0
omega = SA[2.53e-5, 0, 6.62e-5]
alpha = 4e-3 #kg/s
g = SA[-3.71, 0, 0]
model = Rocket{T}(initialmass, omega, alpha, g)

Rocket{Float64}(2000.0, [2.53e-5, 0.0, 6.62e-5], 0.004, [-3.71, 0.0, 0.0])

In [311]:
mf = 300.0

initialthrust = [5,6,7]
initialnorm = norm(initialthrust)
#xf = SA_F64[0, 0, 0, 0, 0, 0, log(mf)]
#x0 = SA_F64[2400, 450, -330, -10, -40, 10, log(initialmass)]

xf = SA[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, mf]
x0 = SA[2400.0, 450.0, -330.0, -10.0, -40.0, 10.0, initialmass]

#initialstate = SA[1,2,3,4,5,6, initialmass]
#initialcontrol = SA[3,4,0]

#println(initialcontrol)
#ẋ = dynamics(model, x0, initialcontrol)

7-element SVector{7, Float64} with indices SOneTo(7):
 2400.0
  450.0
 -330.0
  -10.0
  -40.0
   10.0
 2000.0

In [312]:
xf

7-element SVector{7, Float64} with indices SOneTo(7):
   0.0
   0.0
   0.0
   0.0
   0.0
   0.0
 300.0

In [313]:
println(model)

Rocket{Float64}(2000.0, [2.53e-5, 0.0, 6.62e-5], 0.004, [-3.71, 0.0, 0.0])


In [314]:
# x,u = rand(model)
# dt = 0.1
# #z = KnotPoint(x,u,dt)
# z = KnotPoint(x, u, t, dt)

In [315]:
n,m = size(model)

(7, 4, 7)

In [316]:
#x′ = discrete_dynamics(model, z)

In [317]:
N = 101              # number of time steps (knot points). Should be odd.
tf = 100.0             # total time (sec)
dt = tf / (N-1)      # time step (sec)

1.0

In [318]:
n,m = RD.dims(model)

(7, 4, 7)

In [319]:
# #Create a Cost Function 

# RobotDynamics.@autodiff struct CartpoleCost{T} <: TrajectoryOptimization.CostFunction
#     Q::Vector{T}
# end

# RobotDynamics.state_dim(::CartpoleCost) = 7
# RobotDynamics.control_dim(::CartpoleCost) = 4


# function RobotDynamics.evaluate(cost::CartpoleCost, x, u)
#     xgoal = SA[0, 0, 0]
#     J = norm(cost.Q*x[SA[1,2,3]] - xgoal)
#     return J
# end

# Q = Diagonal(@SVector fill(1,3))

In [320]:
#obj = norm(x[2:3,end] - x_goal[2:3])

In [321]:
#Q  = Diagonal(SA[0.1,0.1,0.01, 0.1,0.1,0.01, 0.1])

Q_new  = Diagonal(SA[0.1,0.1,0.01, 0.1,0.1,0.01, 0.1])
R_new  = Diagonal(SA[0.01, 0.1, 0.1, 0.1])
Qf_new = Diagonal(SA[1e2,1e2,1e3, 1e2,1e2,1e3, 1e3])

obj = LQRObjective(Q_new,R_new,Qf_new,xf,N)

Objective

In [322]:
cons = ConstraintList(n,m,N)

println(m)

4


In [323]:
println(xf)

[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 300.0]


In [324]:
#goal constraint
goal = GoalConstraint(xf, 1:6)
add_constraint!(cons, goal, N)

#add control limits

#ctrl = NormConstraint(n, m, 24000, Inequality(), :control)

#add_constraint!(cons, ctrl, 1:N-1)  # add to all but the last time step

In [325]:
#prob = Problem(model, obj, x0, tf, xf=xf, constraints = cons)



#prob = Problem(model, obj, x0, tf, xf=xf, constraints=cons)

prob = Problem(model, obj, x0, tf, xf=xf, constraints=cons)



Problem{Float64}(RobotDynamics.DiscretizedDynamics{Rocket{Float64}, RobotDynamics.RK4, 11}[RobotDynamics.DiscretizedDynamics{Rocket{Float64}, RobotDynamics.RK4, 11}(Rocket{Float64}(2000.0, [2.53e-5, 0.0, 6.62e-5], 0.004, [-3.71, 0.0, 0.0]), RobotDynamics.RK4(RobotDynamics.ADVector{Float64}([0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0], Dict{DataType, Vector{D} where D<:(ForwardDiff.Dual{Nothing, Float64, N} where N)}()), RobotDynamics.ADVector{Float64}([0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0], Dict{DataType, Vector{D} where D<:(ForwardDiff.Dual{Nothing, Float64, N} where N)}()), RobotDynamics.ADVector{Float64}([0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0], Dict{DataType, Vector{D} where D<:(ForwardDiff.Dual{Nothing, Float64, N} where N)}()), RobotDynamics.ADVector{Float64}([0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0], Dict{DataType, Vector{D} where D<:(ForwardDiff.Dual{Nothing, Float64, N} where N)}()), [[0.0 0.0 … 0.0 0.0; 0.0 0.0 … 0.0 0.0; … ; 0.0 0.0 … 0.0 0.0; 0.0 0.0 … 0.0 0.0], [0.0 0.0 … 0.0 0.0; 0.0 0.0 … 0.0 

In [326]:
constraints = get_constraints(prob)
inds = constraints.inds

1-element Vector{UnitRange{Int64}}:
 101:101

In [327]:
# k=3
# RobotDynamics.state_dim(prob, k)
# RobotDynamics.control_dim(prob, k)
# RobotDynamics.dims(prob, k)

In [328]:
initial_controls!(prob, [SA[3,4,0,5] for k = 1:N-1])

In [329]:
rollout!(prob)

In [330]:
states(prob)

101-element Vector{SubArray{Float64, 1, Vector{Float64}, Tuple{UnitRange{Int64}}, true}}:
 [2400.0, 450.0, -330.0, -10.0, -40.0, 10.0, 2000.0]
 [2388.1431076142835, 410.00199806780444, -319.9989901446846, -13.71378469039561, -39.99592192665818, 10.002019680615712, 1999.98]
 [2372.5724307894193, 370.0083200221407, -309.99596070190785, -17.427568870160165, -39.991352226945594, 10.004039171813629, 1999.96]
 [2353.2879700604444, 330.01945748935117, -299.99091187041495, -21.141352490480855, -39.9862909009198, 10.006058454938817, 1999.94]
 [2330.2897260112095, 290.0359020957182, -289.9838438676061, -24.855135502544876, -39.980737948643274, 10.008077511336348, 1999.92]
 [2303.5776992743777, 250.0581454674592, -279.9747569295364, -28.56891785753943, -39.97469337018344, 10.010096322351293, 1999.9]
 [2273.1518905314247, 210.08667923072178, -269.96365131091596, -32.282699506651724, -39.968157165612645, 10.012114869328727, 1999.88]
 [2239.0123005126393, 170.12199501157863, -259.95052728510973, -35

In [331]:
solver = ALTROSolver(prob)

ALTROSolver{Float64, Altro.iLQRSolver{Vector{RobotDynamics.DiscretizedDynamics{Rocket{Float64}, RobotDynamics.RK4, 11}}, Altro.ALObjective{Float64, Objective{DiagonalCost{7, 4, Float64}}}, 7, 7, 4, Float64, SVector{11, Float64}}, Altro.ProjectedNewtonSolver{RobotDynamics.DiscretizedDynamics{Rocket{Float64}, RobotDynamics.RK4, 11}, Objective{DiagonalCost{7, 4, Float64}}, 7, 4, Float64, RobotDynamics.InPlace}}(SolverOptions{Float64}(1.0e-6, 0.0001, 0.0001, 10.0, 1.0, 1.0e-10, 300, 10, false, 1.0e-8, 10.0, 0.5, 20, 1.0e8, 1.0e8, 1.0e8, true, false, false, false, 0.0, 1.6, 1.0e8, 1.0e-8, :control, 10.0, false, 1.0, 10.0, 1.0e8, 1.0e8, 0.001, 30, false, true, true, false, false, 2, :feasible, 0.001, 0.001, true, 0.01, 1.0e-8, 1.0e-8, 1.1, RobotDynamics.StaticReturn(), RobotDynamics.ForwardAD(), true, false, true, 1000, true, 0), SolverStats{Float64}(0, 0, 0, [0, 0, 0, 0, 0, 0, 0, 0, 0, 0  …  0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0, 0, 0  …  0, 0, 0, 0, 0, 0, 0, 0, 0, 0], Bool

In [332]:
# Set up solver options
opts = SolverOptions()
opts.cost_tolerance = 1e-10

# Create a solver, adding in additional options
solver = ALTROSolver(prob, opts, show_summary=true)

ALTROSolver{Float64, Altro.iLQRSolver{Vector{RobotDynamics.DiscretizedDynamics{Rocket{Float64}, RobotDynamics.RK4, 11}}, Altro.ALObjective{Float64, Objective{DiagonalCost{7, 4, Float64}}}, 7, 7, 4, Float64, SVector{11, Float64}}, Altro.ProjectedNewtonSolver{RobotDynamics.DiscretizedDynamics{Rocket{Float64}, RobotDynamics.RK4, 11}, Objective{DiagonalCost{7, 4, Float64}}, 7, 4, Float64, RobotDynamics.InPlace}}(SolverOptions{Float64}(1.0e-6, 1.0e-10, 0.0001, 10.0, 1.0, 1.0e-10, 300, 10, false, 1.0e-8, 10.0, 0.5, 20, 1.0e8, 1.0e8, 1.0e8, true, false, false, false, 0.0, 1.6, 1.0e8, 1.0e-8, :control, 10.0, false, 1.0, 10.0, 1.0e8, 1.0e8, 0.001, 30, false, true, true, false, false, 2, :feasible, 0.001, 0.001, true, 0.01, 1.0e-8, 1.0e-8, 1.1, RobotDynamics.StaticReturn(), RobotDynamics.ForwardAD(), true, false, true, 1000, true, 0), SolverStats{Float64}(0, 0, 0, [0, 0, 0, 0, 0, 0, 0, 0, 0, 0  …  0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0, 0, 0  …  0, 0, 0, 0, 0, 0, 0, 0, 0, 0], Boo

In [333]:
solve!(solver)


SOLVE COMPLETED
 solved using the ALTRO Solver,
 part of the Altro.jl package developed by the REx Lab at Stanford and Carnegie Mellon Universities

  Solve Statistics
    Total Iterations: 71
    Solve Time: 1568.314109 (ms)

  Covergence
    Terminal Cost: 2.386967352906436e7
    Terminal dJ: -0.2167665809392929
    Terminal gradient: 7.156663702788144e-6
    Terminal constraint violation: 1.463515562714275e-10
    Solve Status: SOLVE_SUCCEEDED


ALTROSolver{Float64, Altro.iLQRSolver{Vector{RobotDynamics.DiscretizedDynamics{Rocket{Float64}, RobotDynamics.RK4, 11}}, Altro.ALObjective{Float64, Objective{DiagonalCost{7, 4, Float64}}}, 7, 7, 4, Float64, SVector{11, Float64}}, Altro.ProjectedNewtonSolver{RobotDynamics.DiscretizedDynamics{Rocket{Float64}, RobotDynamics.RK4, 11}, Objective{DiagonalCost{7, 4, Float64}}, 7, 4, Float64, RobotDynamics.InPlace}}(SolverOptions{Float64}(1.0e-6, 1.0e-10, 0.0001, 10.0, 1.0, 1.0e-10, 300, 10, false, 1.0e-8, 10.0, 0.5, 20, 1.0e8, 1.0e8, 1.0e8, true, false, false, false, 0.0, 1.6, 1.0e8, 1.0e-8, :control, 10.0, false, 1.0, 10.0, 1.0e8, 1.0e8, 0.001, 30, false, true, true, false, false, 2, :feasible, 0.001, 0.001, true, 0.01, 1.0e-8, 1.0e-8, 1.1, RobotDynamics.StaticReturn(), RobotDynamics.ForwardAD(), true, false, true, 1000, true, 0), SolverStats{Float64}(71, 7, 1, [1, 2, 3, 4, 5, 6, 7, 8, 9, 10  …  62, 63, 64, 65, 66, 67, 68, 69, 70, 71], [0, 0, 0, 0, 0, 0, 0, 0, 0, 0  …  5, 5, 5, 5, 5, 6, 6, 6

In [334]:
status(solver)

SOLVE_SUCCEEDED::TerminationStatus = 2

In [336]:
X = states(solver)     # alternatively states(prob)
U = controls(solver)   # alternatively controls(prob)

100-element Vector{SVector{4, Float64}}:
 [-12022.321792379265, 1283.9964108267343, -161.93424388354367, 0.0]
 [-11635.598734425708, 1295.233220222362, -164.40224394851896, 0.0]
 [-11254.747894591275, 1305.8054793412948, -166.84868743284642, 0.0]
 [3431.7766875710745, 1520.8172568647642, -195.65944339430393, 0.0]
 [3707.057279994955, 1481.3775369724394, -191.94267704225425, 0.0]
 [3938.888312917682, 1448.7635842609557, -189.0963475096071, 0.0]
 [4142.330226613795, 1420.407106807279, -186.79947452475645, 0.0]
 [4324.526207769606, 1395.0213016917055, -184.89187686796566, 0.0]
 [4489.50121583918, 1371.8444633951897, -183.27900041690805, 0.0]
 [4639.776606939887, 1350.3784220776438, -181.89910682908118, 0.0]
 [4777.052755299952, 1330.2748288049006, -180.7090477168949, 0.0]
 [4902.542072010314, 1311.2780658651589, -179.67714008290758, 0.0]
 [5017.148461591553, 1293.1935671194703, -178.77922105398144, 0.0]
 ⋮
 [2041.0892678550028, -251.59196314792285, 15.10589743830137, 0.0]
 [2036.418179173

In [337]:
#Obtain optimal x
Xm = hcat(Vector.(X)...)  # convert to normal Vector before concatenating so it's fast
Um = hcat(Vector.(U)...)

4×100 Matrix{Float64}:
 -12022.3    -11635.6    -11254.7    …  1915.59   1855.58   -6603.42
   1284.0      1295.23     1305.81      -661.208  -719.825   -666.324
   -161.934    -164.402    -166.849      132.182   148.966    142.117
      0.0         0.0         0.0          0.0       0.0        0.0

In [335]:
println("Number of iterations: ", iterations(solver))
println("Final cost: ", cost(solver))
println("Final constraint satisfaction: ", max_violation(solver))

Number of iterations: 71
Final cost: 2.386967352906737e7
Final constraint satisfaction: 1.463515562714275e-10


In [ ]:
omega = SA[1,2,3]
A = [zeros(3,3) I; -(skewmatrix(omega))^2 -2*skewmatrix(omega)]

In [ ]:
upper = [(@SMatrix zeros(3,3)) (SMatrix{3,3}(1I))]

In [ ]:
 mid = [-(skewmatrix(omega))^2 -2*skewmatrix(omega)]

In [ ]:
A = [upper;mid]

In [108]:
omega = SA[2.53e-5 0 6.62e-5]
α = 2e-5
u = [2,3,4]
t = 3

upper = [(@SMatrix zeros(3,3)) (SMatrix{3,3}(1I))]
#mid = [(@SMatrix zeros(3,3)) (SMatrix{3,3}(1I)) (@SVector zeros(3)]

mid = [-(skewmatrix(omega))^2 -2*skewmatrix(omega)]
#lower = (@SVector zeros(7))'
    
mdot = -α*norm(u)
    
currentmass = m - mdot*t
    
println("this is current mass: ", currentmass)
    
A = [upper; mid] #makes A static

println("A matrix: ", typeof(A))

    
#B = [SMatrix{3,3}(1I); @SMatrix zeros(3,3); (@SVector zeros(3))'] #make B static
    
B = [@SMatrix zeros(3,3); SMatrix{3,3}(1I)] #make B static
    
d = (g+(u/currentmass))
    
xdot = A*x0 + B*d

this is current mass: 2000.0003231098885
A matrix: SMatrix{6, 6, Float64, 36}


6-element SVector{6, Float64} with indices SOneTo(6):
 -10.0
 -40.0
  10.0
  -3.714284929601755
   0.0033322598961676225
   0.004019768783190164

In [109]:
B = [@SMatrix zeros(3,3); SMatrix{3,3}(1I)] #make B static

6×3 SMatrix{6, 3, Float64, 18} with indices SOneTo(6)×SOneTo(3):
 0.0  0.0  0.0
 0.0  0.0  0.0
 0.0  0.0  0.0
 1.0  0.0  0.0
 0.0  1.0  0.0
 0.0  0.0  1.0

In [110]:
A = [upper; mid] #makes A static

6×6 SMatrix{6, 6, Float64, 36} with indices SOneTo(6)×SOneTo(6):
  0.0          0.0          0.0          1.0         0.0         0.0
  0.0          0.0          0.0          0.0         1.0         0.0
  0.0          0.0          0.0          0.0         0.0         1.0
  4.38244e-9  -0.0         -1.67486e-9  -0.0         0.0001324  -0.0
 -0.0          5.02253e-9  -0.0         -0.0001324  -0.0         5.06e-5
 -1.67486e-9  -0.0          6.4009e-10   0.0        -5.06e-5    -0.0

In [113]:
x0_new = SA[2400, 450, -330, -10, -40, 10, 5]

7-element SVector{7, Int64} with indices SOneTo(7):
 2400
  450
 -330
  -10
  -40
   10
    5

In [115]:
    m = model.m   # intial mass of the rocket
    omega = model.omega   # mass of the pole j(point mass at the end) in kg
    α = model.α     # length of the pole in m
    g = model.g     # gravity m/s^2

    #q  = x[SA[1,2,3]] #position
    #qd = x[SA[4,5,6]] #first derivative is the velocity, last elements of state
    
    upper = [(@SMatrix zeros(3,3)) (SMatrix{3,3}(1I)) (@SVector zeros(3))]
    #mid = [(@SMatrix zeros(3,3)) (SMatrix{3,3}(1I)) (@SVector zeros(3))]
    mid = [(-(skewmatrix(omega))^2) (-2*skewmatrix(omega)) (@SVector zeros(3))]
    lower = (@SVector zeros(7))'
    
    A = [upper; mid; lower] #makes A static
    
    #B = [SMatrix{3,3}(1I); @SMatrix zeros(3,3); (@SVector zeros(3))'] #make B static
    
    B = [@SMatrix zeros(3,4); SMatrix{3,3}(1I) @SVector zeros(3); (@SVector zeros(3))' -α] #make B static
    
    B_2 = [(g+(u[SA[1,2,3]]/x0_new[7])); norm(u[SA[1,2,3]])]
    
    xdot = A*x0_new + B*B_2

    return xdot

7-element SVector{7, Float64} with indices SOneTo(7):
 -10.0
 -40.0
  10.0
  -3.3152849294402
   0.6018322601385
   0.8020197691063
  -0.021540659228538015